This notebook narrates the <a href="https://www.kaggle.com/c/expedia-hotel-recommendations">Kaggle Expedia hotel recommendations competition </a>. I would like to acknowledge Vik Paruchuri for providing the wonderful guide for this competition.

The competition Details can be found at the above link. But I would like to summarize it for quick insight.

<b> The Expedia Kaggle competition</b>

The Expedia competition challenges you with predicting what hotel a user will book based on some attributes about the search the user is conducting on Expedia. Before we dive into any coding, we’ll need to put in time to understand both the problem and the data.

<b> Data Fields </b>

The details of the data fields is available at Kaggle site <a href="https://www.kaggle.com/c/expedia-hotel-recommendations/data"> data details </a>.


<b> Data Exploration </b>


In [1]:
#library imports
import pandas as pd
import numpy as np
import ml_metrics as metrics

In [2]:
destinations = pd.read_csv("destinations.csv")
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

In [3]:
#lets see the size of the different data files
print("train data size:",train.shape)
print("test data size:",test.shape)

train data size: (37670293, 24)
test data size: (2528243, 22)


We have about 37 million training set rows, and 2 million testing set rows, which will make this problem a bit challenging to work with.

We can explore the first few rows of the data:

In [4]:
train.head() #displays first 5 rows by default

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,...,0,1,8250,1,0,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,...,0,1,14984,1,0,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,...,0,1,14984,1,0,1,2,50,1457,21


There are a few things that immediately stick out:

<ul>
<li> date_time could be useful in our predictions, so we’ll need to convert it.
<li> Most of the columns are integers or floats, so we can’t do a lot of feature engineering. For example, user_location_country isn’t the name of a country, it’s an integer. This makes it harder to create new features, because we don’t know exactly which each value means.
</ul>

In [5]:
test.head()

,id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,...,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market
0,0,2015-09-03 17:09:54,2,3,66,174,37449,5539.0567,1,1,...,2016-05-19,2016-05-23,2,0,1,12243,6,6,204,27
1,1,2015-09-24 17:38:35,2,3,66,174,37449,5873.2923,1,1,...,2016-05-12,2016-05-15,2,0,1,14474,7,6,204,1540
2,2,2015-06-07 15:53:02,2,3,66,142,17440,3975.9776,20,0,...,2015-07-26,2015-07-27,4,0,1,11353,1,2,50,699
3,3,2015-09-14 14:49:10,2,3,66,258,34156,1508.5975,28,0,...,2015-09-14,2015-09-16,2,0,1,8250,1,2,50,628
4,4,2015-07-17 09:32:04,2,3,66,467,36345,66.7913,50,0,...,2015-07-22,2015-07-23,2,0,1,11812,1,2,50,538


There are a few things we can take away from looking at test.csv:

<ul>
<li> It looks like all the dates in test.csv are later than the dates in train.csv, and the <a href="https://www.kaggle.com/c/expedia-hotel-recommendations/data">data page</a> confirms this. The testing set contains dates from 2015, and the training set contains dates from 2013 and 2014.
<li> It looks like the user ids in test.csv are a subset of the user ids in train.csv, given the overlapping integer ranges. We can confirm this later on.
<li> The is_booking column always looks to be 1 in test.csv. The <a href="https://www.kaggle.com/c/expedia-hotel-recommendations/data">data page</a> confirms this.
</ul>


<b> What to predict </b>


We’ll be predicting which hotel_cluster a user will book after a given search. According to the description, there are 100 clusters in total.


<b> How we’ll be scored </b>

The evaluation page says that we’ll be scored using Mean Average Precision @ 5, which means that we’ll need to make 5 cluster predictions for each row, and will be scored on whether or not the correct prediction appears in our list. If the correct prediction comes earlier in the list, we get more points.

For example, if the “correct” cluster is 3, and we predict 4, 43, 60, 3, 20, our score will be lower than if we predict 3, 4, 43, 60, 20. We should put predictions we’re more certain about earlier in our list of predictions.


<b> Exploring Hotel Clusters </b>


In [6]:
train["hotel_cluster"].value_counts()

91    1043720
41     772743
48     754033
64     704734
65     670960
5      620194
98     589178
59     570291
42     551605
21     550092
70     545572
18     545284
83     534132
46     534038
25     530591
62     518809
95     509266
28     507016
68     503797
82     503755
37     496061
50     489892
30     489287
9      488328
58     483253
97     479446
16     477868
72     457463
1      452694
99     444887
       ...   
19     282893
84     278264
66     273505
38     269246
87     260398
23     259233
12     259022
31     257587
67     255946
43     253578
7      252447
54     250745
92     244343
89     243560
45     241408
49     240124
3      225250
80     220218
60     217919
71     216054
93     214293
86     209054
14     192299
75     165226
24     164127
35     139122
53     134812
88     107784
27     105040
74      48355
Name: hotel_cluster, Length: 100, dtype: int64

The output above is truncated, but it shows that the number of hotels in each cluster is fairly evenly distributed. There doesn’t appear to be any relationship between cluster number and the number of items.

<b>Exploring train and test user ids</b>


Finally, we’ll confirm our hypothesis that all the test user ids are found in the train DataFrame. We can do this by finding the unique values for user_id in test, and seeing if they all exist in train.

In [7]:
#find the intersection of train and test user ids
test_ids = set(test["user_id"].unique())
train_ids = set(train.user_id.unique())
intersection_count = len(test_ids & train_ids)
intersection_count == len(test_ids)

True

In [8]:
#another way to do this using numpy array
a = np.array(list(test_ids)) # donot know why in jupyter I need to cast the set to list before converting to array, in python console a set can be directly converted to array
b = np.array(list(train_ids))
common_users = np.intersect1d(a, b)
print(len(common_users), len(test_ids))
len(common_users) == len(test_ids)

1181577 1181577


True

Looks like our hypothesis is correct, which will make working with this data much easier!

<b> Downsampling our Kaggle data </b>


The entire train.csv dataset contains 37 million rows, which makes it hard to experiment with different techniques. Ideally, we want a small enough dataset that lets us quickly iterate through different approaches but is still representative of the whole training data.

We can do this by first randomly sampling rows from our data, then selecting new training and testing datasets from train.csv. By selecting both sets from train.csv, we’ll have the true hotel_cluster label for every row, and we’ll be able to calculate our accuracy as we test techniques

<b> Add in times and dates</b>

The first step is to add month and year fields to train. Because the train and test data is differentiated by date, we’ll need to add date fields to allow us to segment our data into two sets the same way. If we add year and month fields, we can split our data into training and testing sets using them.

The code below will:
<ul>
<li> Convert the date_time column in train from an object to a datetime value. This makes it easier to work with as a date.
<li> Extract the year and month from from date_time, and assign them to their own columns.
</ul>

In [9]:
train["date_time"] = pd.to_datetime(train["date_time"])
train["year"] = train["date_time"].dt.year
train["month"] = train["date_time"].dt.month

<b> Pick 10000 users</b>

Because the user ids in test are a subset of the user ids in train, we’ll need to do our random sampling in a way that preserves the full data of each user. We can accomplish this by selecting a certain number of users randomly, then only picking rows from train where user_id is in our random sample of user ids.


In [10]:
import random

unique_user_ids = train["user_id"].unique()
sel_user_ids = random.sample(set(unique_user_ids),10000)
sel_train = train[train.user_id.isin(sel_user_ids)]

<b> Pick new training and testing sets </b>

We’ll now need to pick new training and testing sets from sel_train. We’ll call these sets t1 and t2.


In [11]:
t1 = sel_train[((sel_train.year == 2013) | ((sel_train.year == 2014) & (sel_train.month < 8)))]
t2 = sel_train[((sel_train.year == 2014) & (sel_train.month >= 8))]


In the original train and test DataFrames, test contained data from 2015, and train contained data from 2013 and 2014. We split this data so that anything after July 2014 is in t2, and anything before is in t1. This gives us smaller training and testing sets with similar characteristics to train and test.

<b> Remove click events</b>

If is_booking is 0, it represents a click, and a 1 represents a booking.  test contains only booking events, so we’ll need to sample t2 to only contain bookings as well.

In [12]:
t2 = t2[t2.is_booking == 1]

<b> A simple algorithm</b>


The most simple technique we could try on this data is to find the most common clusters across the data, then use them as predictions. (Whichever cluster has many user_ids will be assumed to be the WINNER!!!)

We can again use the value_counts method to help us here:

In [13]:
#lets take few of the top clusters
most_common_clusters = list(train.hotel_cluster.value_counts().head().index)



The above code will give us a list of the 5 most common clusters in train. This is because the head method returns the first 5 rows by default, and the index property will return the index of the DataFrame, which is the hotel cluster after running the value_counts method.

<b>Generating predictions</b>


We can turn most_common_clusters into a list of predictions by making the same prediction for each row.



In [14]:
predictions = [most_common_clusters for i in range(t2.shape[0])]
#print(predictions[0])
#print(len(predictions), t2.shape)

This will create a list with as many elements as there are rows in t2. Each element will be equal to most_common_clusters.

<b> Evaluating error</b>


In order to evaluate error, we’ll first need to figure out how to compute Mean Average Precision. Luckily, <a href="https://github.com/benhamner">Ben Hamner</a> has written an implementation that can be found <a href="https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py">here</a>. It can be installed as part of the ml_metrics package, and you can find installation instructions for how to install it here.

We can compute our error metric with the mapk method in ml_metrics. 

In [15]:
target = [[l] for l in t2["hotel_cluster"]]
#metrics.mapk(target, predictions, k=5)
#the mapk takes list of lists as the target and predictions
print(metrics.mapk(target, predictions, k=5))

0.0631348332926


Our target needs to be in list of lists format for mapk to work, so we convert the hotel_cluster column of t2 into a list of lists. Then, we call the mapk method with our target, our predictions, and the number of predictions we want to evaluate (5).

Our result here isn’t great, but we’ve just generated our first set of predictions, and evaluated our error! The framework we’ve built will allow us to quickly test out a variety of techniques and see how they score. We’re well on our way to building a good-performing solution for the leaderboard.

<b> Finding correlations </b>


Before we move on to creating a better algorithm, let’s see if anything correlates well with hotel_cluster. This will tell us if we should dive more into any particular columns.

We can find linear correlations in the training set using the corr method:

In [16]:
train.corr()["hotel_cluster"]

site_name                   -0.022408
posa_continent               0.014938
user_location_country       -0.010477
user_location_region         0.007453
user_location_city           0.000831
orig_destination_distance    0.007260
user_id                      0.001052
is_mobile                    0.008412
is_package                   0.038733
channel                      0.000707
srch_adults_cnt              0.012309
srch_children_cnt            0.016261
srch_rm_cnt                 -0.005954
srch_destination_id         -0.011712
srch_destination_type_id    -0.032850
is_booking                  -0.021548
cnt                          0.002944
hotel_continent             -0.013963
hotel_country               -0.024289
hotel_market                 0.034205
hotel_cluster                1.000000
year                        -0.001050
month                       -0.000560
Name: hotel_cluster, dtype: float64

This tells us that no columns correlate linearly with hotel_cluster. This makes sense, because there is no linear ordering to hotel_cluster. For example, having a higher cluster number isn’t tied to having a higher srch_destination_id.

Unfortunately, this means that techniques like linear regression and logistic regression won’t work well on our data, because they rely on linear correlations between predictors and targets.



<b> Creating better predictions for our Kaggle entry</b>

This data for this competition is quite difficult to make predictions on using machine learning for a few reasons:

<ul>

<li> There are millions of rows, which increases runtime and memory usage for algorithms.

<li> There are 100 different clusters, and according to the competition admins, the boundaries are fairly fuzzy, so it will likely be hard to make predictions. As the number of clusters increases, classifiers generally decrease in accuracy.

<li> Nothing is linearly correlated with the target (hotel_clusters), meaning we can’t use fast machine learning techniques like linear regression.


For these reasons, machine learning probably won’t work well on our data, but we can try an algorithm and find out.

<b> Generating features</b>

The first step in applying machine learning is to generate features. We can generate features using both what’s available in the training data, and what’s available in destinations. We haven’t looked at destinations yet, so let’s take a quick peek.

<b> Generating features from destinations</b>

Destinations contains an id that corresponds to srch_destination_id, along with 149 columns of latent information about that destination. Here’s a sample:

In [17]:
destinations.head()

,srch_destination_id,d1,d2,d3,d4,d5,d6,d7,d8,d9,...,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
0,0,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-1.897627,-2.198657,-2.198657,-1.897627,...,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657
1,1,-2.181690,-2.181690,-2.181690,-2.082564,-2.181690,-2.165028,-2.181690,-2.181690,-2.031597,...,-2.165028,-2.181690,-2.165028,-2.181690,-2.181690,-2.165028,-2.181690,-2.181690,-2.181690,-2.181690
2,2,-2.183490,-2.224164,-2.224164,-2.189562,-2.105819,-2.075407,-2.224164,-2.118483,-2.140393,...,-2.224164,-2.224164,-2.196379,-2.224164,-2.192009,-2.224164,-2.224164,-2.224164,-2.224164,-2.057548
3,3,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.115485,-2.177409,-2.177409,-2.177409,...,-2.161081,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409
4,4,-2.189562,-2.187783,-2.194008,-2.171153,-2.152303,-2.056618,-2.194008,-2.194008,-2.145911,...,-2.187356,-2.194008,-2.191779,-2.194008,-2.194008,-2.185161,-2.194008,-2.194008,-2.194008,-2.188037


The competition doesn’t tell us exactly what each latent feature is, but it’s safe to assume that it’s some combination of destination characteristics, like name, description, and more. These latent features were converted to numbers, so they could be anonymized.

We can use the destination information as features in a machine learning algorithm, but we’ll need to compress the number of columns down first, to minimize runtime. We can use PCA to do this. PCA will reduce the number of columns in a matrix while trying to preserve the same amount of variance per row. Ideally, PCA will compress all the information contained in all the columns into less, but in practice, some information is lost.

In the code below, we:

<ul>
<li> Initialize a PCA model using scikit-learn.
<li> Specify that we want to only have 3 columns in our data.
<li> Transform the columns d1-d149 into 3 columns.

</ul>

In [18]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
# we skip the srch_destination_id from dimensionality reduction because we might need to extract it later
dest_small = pca.fit_transform(destinations[["d{0}".format(i + 1) for i in range(149)]])
dest_small = pd.DataFrame(dest_small)
#now we apply the srch_destination_id to our dimension reduced data
dest_small["srch_destination_id"] = destinations["srch_destination_id"]
#lets see the column names
print(dest_small.columns.values)

[0 1 2 'srch_destination_id']


The above code compresses the 149 columns in destinations down to 3 columns, and creates a new DataFrame called dest_small. We preserve most of the variance in destinations while doing this, so we don’t lose a lot of information, but save a lot of runtime for a machine learning algorithm.

<b>Generating features</b>

Now that the preliminaries are done with, we can generate our features. We’ll do the following:

<ul>

<li> Generate new date features based on date_time, srch_ci, and srch_co.
<li> Remove non-numeric columns like date_time.
<li> Add in features from dest_small.
<li> Replace any missing values with -1.
</ul

In [19]:
#lets review the columns in train and test set
print(train.columns.values)
print(test.columns.values)

['date_time' 'site_name' 'posa_continent' 'user_location_country'
 'user_location_region' 'user_location_city' 'orig_destination_distance'
 'user_id' 'is_mobile' 'is_package' 'channel' 'srch_ci' 'srch_co'
 'srch_adults_cnt' 'srch_children_cnt' 'srch_rm_cnt' 'srch_destination_id'
 'srch_destination_type_id' 'is_booking' 'cnt' 'hotel_continent'
 'hotel_country' 'hotel_market' 'hotel_cluster' 'year' 'month']
['id' 'date_time' 'site_name' 'posa_continent' 'user_location_country'
 'user_location_region' 'user_location_city' 'orig_destination_distance'
 'user_id' 'is_mobile' 'is_package' 'channel' 'srch_ci' 'srch_co'
 'srch_adults_cnt' 'srch_children_cnt' 'srch_rm_cnt' 'srch_destination_id'
 'srch_destination_type_id' 'hotel_continent' 'hotel_country'
 'hotel_market']


In [20]:
def calc_fast_features(df):
    df["date_time"] = pd.to_datetime(df["date_time"])
    df["srch_ci"] = pd.to_datetime(df["srch_ci"], format='%Y-%m-%d', errors="coerce")
    df["srch_co"] = pd.to_datetime(df["srch_co"], format='%Y-%m-%d', errors="coerce")
    
    props = {}
    for prop in ["month", "day", "hour", "minute", "dayofweek", "quarter"]:
        props[prop] = getattr(df["date_time"].dt, prop)
    
    carryover = [p for p in df.columns if p not in ["date_time", "srch_ci", "srch_co"]]
    for prop in carryover:
        props[prop] = df[prop]
    
    date_props = ["month", "day", "dayofweek", "quarter"]
    for prop in date_props:
        props["ci_{0}".format(prop)] = getattr(df["srch_ci"].dt, prop)
        props["co_{0}".format(prop)] = getattr(df["srch_co"].dt, prop)
    props["stay_span"] = (df["srch_co"] - df["srch_ci"]).astype('timedelta64[h]')
        
    ret = pd.DataFrame(props)
    
    ret = ret.join(dest_small, on="srch_destination_id", how='left', rsuffix="dest")
    ret = ret.drop("srch_destination_iddest", axis=1)
    return ret

df = calc_fast_features(t1)
df.fillna(-1, inplace=True)
print(df.columns.values)

C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarn

['channel' 'ci_day' 'ci_dayofweek' 'ci_month' 'ci_quarter' 'cnt' 'co_day'
 'co_dayofweek' 'co_month' 'co_quarter' 'day' 'dayofweek' 'hotel_cluster'
 'hotel_continent' 'hotel_country' 'hotel_market' 'hour' 'is_booking'
 'is_mobile' 'is_package' 'minute' 'month' 'orig_destination_distance'
 'posa_continent' 'quarter' 'site_name' 'srch_adults_cnt'
 'srch_children_cnt' 'srch_destination_id' 'srch_destination_type_id'
 'srch_rm_cnt' 'stay_span' 'user_id' 'user_location_city'
 'user_location_country' 'user_location_region' 'year' 0 1 2]


The above will calculate features such as length of stay, check in day, and check out month. These features will help us train a machine learning algorithm later on.

Replacing missing values with -1 isn’t the best choice, but it will work fine for now, and we can always optimize the behavior later on.


<b> Machine learning</b>

Now that we have features for our training data, we can try machine learning. We’ll use 3-fold cross validation across the training set to generate a reliable error estimate. Cross validation splits the training set up into 3 parts, then predicts hotel_cluster for each part using the other parts to train with.

We’ll generate predictions using the Random Forest algorithm. Random forests build trees, which can fit to nonlinear tendencies in data. This will enable us to make predictions, even though none of our columns are linearly related.

We’ll first initialize the model and compute cross validation scores:


In [21]:
predictors = [c for c in df.columns if c not in ["hotel_cluster"]]
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=10, min_weight_fraction_leaf=0.1)
scores = cross_validation.cross_val_score(clf, df[predictors], df['hotel_cluster'], cv=3)
scores

C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


array([ 0.06822777,  0.07206147,  0.06842344])

The above code doesn’t give us very good accuracy, and confirms our original suspicion that machine learning isn’t a great approach to this problem. However, classifiers tend to have lower accuracy when there is a high cluster count. We can instead try training 100 binary classifiers. Each classifier will just determine if a row is in it’s cluster, or not. This will entail training one classifier per label in hotel_cluster.

<b> Binary classifiers</b>

We’ll again train Random Forests, but each forest will predict only a single hotel cluster. We’ll use 2 fold cross validation for speed, and only train 10 trees per label.

In the code below, we:

<ul>

<li> Loop across each unique hotel_cluster.
<ul>
    <li> Train a Random Forest classifier using 2-fold cross validation.
    <li> Extract the probabilities from the classifier that the row is in the unique hotel_cluster
</ul>
<li> Combine all the probabilities.
<li> For each row, find the 5 largest probabilities, and assign those hotel_cluster values as predictions.
<li> Compute accuracy using mapk.
</ul>

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold
from itertools import chain

all_probs = []
unique_clusters = df["hotel_cluster"].unique()
for cluster in unique_clusters:
    df["target"] = 1
    df["target"][df["hotel_cluster"] != cluster] = 0
    predictors = [col for col in df if col not in ['hotel_cluster', "target"]]
    probs = []
    cv = KFold(len(df["target"]), n_folds=2)
    clf = RandomForestClassifier(n_estimators=10, min_weight_fraction_leaf=0.1)
    for i, (tr, te) in enumerate(cv):
        #print(tr)
        #print(te)
        #print(df[predictors].iloc[tr])
        clf.fit(df[predictors].iloc[tr], df["target"].iloc[tr])
        preds = clf.predict_proba(df[predictors].iloc[te])
        probs.append([p[1] for p in preds])
    full_probs = chain.from_iterable(probs)
    all_probs.append(list(full_probs))

prediction_frame = pd.DataFrame(all_probs).T
prediction_frame.columns = unique_clusters
def find_top_5(row):
    return list(row.nlargest(5).index)

preds = []
for index, row in prediction_frame.iterrows():
    preds.append(find_top_5(row))

#print(t2.columns.values)
#print(t1.columns.values)
metrics.mapk([[l] for l in t2["hotel_cluster"]], preds, k=5)

C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/panda

C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/panda

C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/panda

C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/panda

C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\rbara012.AD.004\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/panda

0.045881226053639847

This performed even worse than before. So, we need to think of something else to get better performance.

<b> Top clusters based on hotel_cluster</b> 

There are a few <a href="https://www.kaggle.com/c/expedia-hotel-recommendations/scripts">Kaggle Scripts</a> for the competition that involve aggregating hotel_cluster based on orig_destination_distance, or srch_destination_id. Aggregating on orig_destination_distance will exploit a data leak in the competition, and attempt to match the same user together. Aggregating on srch_destination_id will find the most popular hotel clusters for each destination. We’ll then be able to predict that a user who searches for a destination is going to one of the most popular hotel clusters for that destination. Think of this as a more granular version of the most common clusters technique we used earlier.

We can first generate scores for each hotel_cluster in each srch_destination_id. We’ll weight bookings higher than clicks. This is because the test data is all booking data, and this is what we want to predict. We want to include click information, but downweight it to reflect this. Step by step, we’ll:

<ul>
<li> Group t1 by srch_destination_id, and hotel_cluster.
<li> Iterate through each group, and:
    <ul>
    <li> Assign 1 point to each hotel cluster where is_booking is True.
    <li> Assign .15 points to each hotel cluster where is_booking is False.
    <li> Assign the score to the srch_destination_id / hotel_cluster combination in a dictionary.
    </ul>

</ul>

Here’s the code to accomplish the above steps:

In [45]:
def make_key(items):
    #print("making key for",str(items))
    return "_".join([str(i) for i in items])

match_cols = ["srch_destination_id"]
cluster_cols = match_cols + ['hotel_cluster']
groups = t1.groupby(cluster_cols)
top_clusters = {}
for name, group in groups:
    clicks = len(group.is_booking[group.is_booking == False])
    bookings = len(group.is_booking[group.is_booking == True])
    
    score = bookings + .15 * clicks
    
    clus_name = make_key(name[:len(match_cols)])
    if clus_name not in top_clusters:
        top_clusters[clus_name] = {}
    top_clusters[clus_name][name[-1]] = score

At the end, we’ll have a dictionary where each key is an srch_destination_id. Each value in the dictionary will be another dictionary, containing hotel clusters as keys with scores as values. Here’s how it looks:

In [46]:
for clus_name in top_clusters:
    clus_dict = top_clusters[clus_name]
    print("\nfor srch_destination_id ",str(clus_name))
    for item in clus_dict:
        print("hotel cluster:",item, " cluster score:", clus_dict[item])


for srch_destination_id  4529
hotel cluster: 42  cluster score: 0.3

for srch_destination_id  26036
hotel cluster: 16  cluster score: 0.3
hotel cluster: 33  cluster score: 0.3
hotel cluster: 48  cluster score: 0.15
hotel cluster: 90  cluster score: 0.15
hotel cluster: 91  cluster score: 0.44999999999999996
hotel cluster: 28  cluster score: 0.15

for srch_destination_id  59377
hotel cluster: 94  cluster score: 1.15

for srch_destination_id  6535
hotel cluster: 14  cluster score: 0.15

for srch_destination_id  59215
hotel cluster: 16  cluster score: 1.15

for srch_destination_id  5683
hotel cluster: 13  cluster score: 0.3

for srch_destination_id  28586
hotel cluster: 90  cluster score: 0.3
hotel cluster: 5  cluster score: 0.15

for srch_destination_id  33439
hotel cluster: 41  cluster score: 0.15
hotel cluster: 42  cluster score: 0.15

for srch_destination_id  6481
hotel cluster: 28  cluster score: 0.3

for srch_destination_id  25156
hotel cluster: 48  cluster score: 1.3

for srch_dest

hotel cluster: 33  cluster score: 0.15
hotel cluster: 3  cluster score: 0.15
hotel cluster: 6  cluster score: 0.15
hotel cluster: 42  cluster score: 0.15
hotel cluster: 28  cluster score: 1.15
hotel cluster: 13  cluster score: 0.15

for srch_destination_id  17739
hotel cluster: 67  cluster score: 0.15

for srch_destination_id  12009
hotel cluster: 96  cluster score: 1.05
hotel cluster: 2  cluster score: 1.65
hotel cluster: 5  cluster score: 0.15
hotel cluster: 72  cluster score: 0.15
hotel cluster: 73  cluster score: 0.6
hotel cluster: 76  cluster score: 1.75
hotel cluster: 13  cluster score: 0.15
hotel cluster: 77  cluster score: 3.9
hotel cluster: 15  cluster score: 0.44999999999999996
hotel cluster: 16  cluster score: 0.6
hotel cluster: 92  cluster score: 0.15
hotel cluster: 18  cluster score: 0.15
hotel cluster: 23  cluster score: 0.3
hotel cluster: 26  cluster score: 0.15
hotel cluster: 91  cluster score: 5.6499999999999995
hotel cluster: 28  cluster score: 2.95
hotel cluster: 93 

hotel cluster: 67  cluster score: 0.15
hotel cluster: 58  cluster score: 0.8999999999999999
hotel cluster: 29  cluster score: 1.15
hotel cluster: 46  cluster score: 1.75
hotel cluster: 61  cluster score: 0.15

for srch_destination_id  33121
hotel cluster: 91  cluster score: 0.3

for srch_destination_id  20531
hotel cluster: 81  cluster score: 0.15
hotel cluster: 82  cluster score: 0.15
hotel cluster: 85  cluster score: 1.15
hotel cluster: 38  cluster score: 1.6

for srch_destination_id  40234
hotel cluster: 5  cluster score: 0.3

for srch_destination_id  28999
hotel cluster: 72  cluster score: 0.15

for srch_destination_id  12699
hotel cluster: 78  cluster score: 1.15
hotel cluster: 61  cluster score: 0.15
hotel cluster: 30  cluster score: 0.15

for srch_destination_id  39796
hotel cluster: 85  cluster score: 1.15

for srch_destination_id  6190
hotel cluster: 16  cluster score: 0.15
hotel cluster: 91  cluster score: 0.3

for srch_destination_id  4386
hotel cluster: 40  cluster score: 0

hotel cluster: 3  cluster score: 1.95
hotel cluster: 5  cluster score: 13.95
hotel cluster: 6  cluster score: 3.1
hotel cluster: 7  cluster score: 0.6
hotel cluster: 10  cluster score: 3.25
hotel cluster: 11  cluster score: 0.6
hotel cluster: 12  cluster score: 2.05
hotel cluster: 14  cluster score: 2.3499999999999996
hotel cluster: 15  cluster score: 2.95
hotel cluster: 16  cluster score: 2.1
hotel cluster: 20  cluster score: 0.75
hotel cluster: 28  cluster score: 4.4
hotel cluster: 29  cluster score: 30.65
hotel cluster: 30  cluster score: 2.4
hotel cluster: 32  cluster score: 0.15
hotel cluster: 35  cluster score: 0.15
hotel cluster: 36  cluster score: 0.6
hotel cluster: 37  cluster score: 0.75
hotel cluster: 40  cluster score: 0.15
hotel cluster: 41  cluster score: 0.44999999999999996
hotel cluster: 42  cluster score: 0.75
hotel cluster: 43  cluster score: 0.3
hotel cluster: 46  cluster score: 25.25
hotel cluster: 47  cluster score: 2.05
hotel cluster: 48  cluster score: 1.9
hotel 

hotel cluster: 46  cluster score: 1.45
hotel cluster: 47  cluster score: 0.15

for srch_destination_id  12659
hotel cluster: 91  cluster score: 1.3
hotel cluster: 68  cluster score: 0.15

for srch_destination_id  20128
hotel cluster: 89  cluster score: 0.15
hotel cluster: 93  cluster score: 0.3

for srch_destination_id  8802
hotel cluster: 0  cluster score: 1.75
hotel cluster: 96  cluster score: 2.2
hotel cluster: 68  cluster score: 0.15
hotel cluster: 5  cluster score: 0.3
hotel cluster: 72  cluster score: 0.6
hotel cluster: 9  cluster score: 3.2
hotel cluster: 10  cluster score: 1.6
hotel cluster: 75  cluster score: 0.15
hotel cluster: 77  cluster score: 1.15
hotel cluster: 16  cluster score: 1.3
hotel cluster: 17  cluster score: 0.3
hotel cluster: 83  cluster score: 0.8999999999999999
hotel cluster: 84  cluster score: 3.1
hotel cluster: 22  cluster score: 0.3
hotel cluster: 89  cluster score: 0.3
hotel cluster: 90  cluster score: 1.05
hotel cluster: 28  cluster score: 2.6
hotel clus

...

hotel cluster: 51  cluster score: 0.3
hotel cluster: 57  cluster score: 2.5
hotel cluster: 58  cluster score: 0.15
hotel cluster: 61  cluster score: 1.05
hotel cluster: 62  cluster score: 0.75
hotel cluster: 63  cluster score: 1.3499999999999999

for srch_destination_id  54096
hotel cluster: 50  cluster score: 0.15

for srch_destination_id  4406
hotel cluster: 40  cluster score: 0.15
hotel cluster: 48  cluster score: 0.3
hotel cluster: 39  cluster score: 1.45

for srch_destination_id  44863
hotel cluster: 48  cluster score: 0.15
hotel cluster: 18  cluster score: 0.15
hotel cluster: 6  cluster score: 0.15
hotel cluster: 9  cluster score: 0.15
hotel cluster: 91  cluster score: 0.6
hotel cluster: 47  cluster score: 0.3

for srch_destination_id  23414
hotel cluster: 32  cluster score: 0.15
hotel cluster: 37  cluster score: 0.15
hotel cluster: 6  cluster score: 1.05
hotel cluster: 7  cluster score: 1.6
hotel cluster: 40  cluster score: 0.15
hotel cluster: 42  cluster score: 0.6
hotel cluste

hotel cluster: 43  cluster score: 1.45
hotel cluster: 60  cluster score: 0.15
hotel cluster: 61  cluster score: 0.6
hotel cluster: 30  cluster score: 0.15
hotel cluster: 15  cluster score: 2.6

for srch_destination_id  40238
hotel cluster: 38  cluster score: 0.3

for srch_destination_id  26604
hotel cluster: 91  cluster score: 0.75
hotel cluster: 37  cluster score: 1.3
hotel cluster: 73  cluster score: 0.44999999999999996
hotel cluster: 42  cluster score: 2.3
hotel cluster: 76  cluster score: 0.15
hotel cluster: 77  cluster score: 0.15
hotel cluster: 15  cluster score: 0.15
hotel cluster: 48  cluster score: 3.5
hotel cluster: 50  cluster score: 0.3
hotel cluster: 90  cluster score: 0.15
hotel cluster: 47  cluster score: 0.6
hotel cluster: 5  cluster score: 0.3

for srch_destination_id  57259
hotel cluster: 81  cluster score: 0.15
hotel cluster: 43  cluster score: 0.15

for srch_destination_id  6934
hotel cluster: 58  cluster score: 0.15

for srch_destination_id  54517
hotel cluster: 33

hotel cluster: 91  cluster score: 3.3499999999999996
hotel cluster: 28  cluster score: 1.15
hotel cluster: 14  cluster score: 0.15
hotel cluster: 47  cluster score: 0.15

for srch_destination_id  12945
hotel cluster: 72  cluster score: 1.15
hotel cluster: 59  cluster score: 0.15
hotel cluster: 21  cluster score: 1.15

for srch_destination_id  11975
hotel cluster: 5  cluster score: 0.44999999999999996
hotel cluster: 8  cluster score: 0.6
hotel cluster: 75  cluster score: 0.15
hotel cluster: 76  cluster score: 2.3
hotel cluster: 13  cluster score: 0.75
hotel cluster: 77  cluster score: 0.3
hotel cluster: 19  cluster score: 0.44999999999999996
hotel cluster: 90  cluster score: 0.44999999999999996
hotel cluster: 91  cluster score: 1.6
hotel cluster: 28  cluster score: 0.75
hotel cluster: 93  cluster score: 0.15
hotel cluster: 30  cluster score: 1.45
hotel cluster: 32  cluster score: 0.44999999999999996
hotel cluster: 33  cluster score: 0.75
hotel cluster: 39  cluster score: 0.3
hotel clust

for srch_destination_id  55375
hotel cluster: 76  cluster score: 0.3
hotel cluster: 60  cluster score: 0.3
hotel cluster: 15  cluster score: 0.3

for srch_destination_id  6440
hotel cluster: 81  cluster score: 0.15

for srch_destination_id  24538
hotel cluster: 6  cluster score: 5.15
hotel cluster: 7  cluster score: 0.6
hotel cluster: 10  cluster score: 0.8999999999999999
hotel cluster: 76  cluster score: 0.15
hotel cluster: 77  cluster score: 1.2
hotel cluster: 14  cluster score: 0.6
hotel cluster: 13  cluster score: 0.44999999999999996
hotel cluster: 17  cluster score: 0.8999999999999999
hotel cluster: 18  cluster score: 0.3
hotel cluster: 83  cluster score: 0.15
hotel cluster: 89  cluster score: 0.44999999999999996
hotel cluster: 28  cluster score: 2.5
hotel cluster: 93  cluster score: 0.75
hotel cluster: 31  cluster score: 0.6
hotel cluster: 39  cluster score: 0.44999999999999996
hotel cluster: 40  cluster score: 0.75
hotel cluster: 42  cluster score: 0.44999999999999996
hotel clus

hotel cluster: 48  cluster score: 0.15

for srch_destination_id  25415
hotel cluster: 81  cluster score: 0.15
hotel cluster: 2  cluster score: 1.15
hotel cluster: 85  cluster score: 0.15
hotel cluster: 82  cluster score: 1.3
hotel cluster: 30  cluster score: 0.15
hotel cluster: 93  cluster score: 0.15

for srch_destination_id  3725
hotel cluster: 57  cluster score: 0.15
hotel cluster: 30  cluster score: 0.15

for srch_destination_id  32041
hotel cluster: 36  cluster score: 3.45

for srch_destination_id  30049
hotel cluster: 48  cluster score: 1.3

for srch_destination_id  58192
hotel cluster: 41  cluster score: 0.3

for srch_destination_id  25278
hotel cluster: 50  cluster score: 1.15
hotel cluster: 6  cluster score: 0.15

for srch_destination_id  778
hotel cluster: 47  cluster score: 1.15

for srch_destination_id  20313
hotel cluster: 84  cluster score: 0.75
hotel cluster: 22  cluster score: 1.45
hotel cluster: 25  cluster score: 0.44999999999999996
hotel cluster: 26  cluster score: 1

We’ll next want to transform this dictionary to find the top 5 hotel clusters for each srch_destination_id. In order to do this, we’ll:

<ul>
<li> Loop through each key in top_clusters.
<li> Find the top 5 clusters for that key.
<li> Assign the top 5 clusters to a new dictionary, cluster_dict.
</ul>

In [47]:
import operator

cluster_dict = {}
for n in top_clusters:
    tc = top_clusters[n]
    top = [l[0] for l in sorted(tc.items(), key=operator.itemgetter(1), reverse=True)[:5]]
    cluster_dict[n] = top

Making predictions based on destination
Once we know the top clusters for each srch_destination_id, we can quickly make predictions. To make predictions, all we have to do is:
<ul>
<li> Iterate through each row in t2.
<li> Extract the srch_destination_id for the row.
<li> Find the top clusters for that destination id.
<li> Append the top clusters to preds.
</ul>

In [48]:
preds = []
for index, row in t2.iterrows():
    key = make_key([row[m] for m in match_cols])
    if key in cluster_dict:
        preds.append(cluster_dict[key])
    else:
        preds.append([])

for item in preds:
    print(str(item))

[60, 30]
[29, 85, 97, 25, 59]
[9, 82, 62, 28, 99]
[97, 9, 64, 85, 83]
[7, 91, 19, 89, 59]
[16, 42, 51, 6, 40]
[16, 34]
[28, 21, 4, 19, 94]
[64, 29, 46, 58, 2]
[64, 2, 81, 3, 46]
[4, 59, 49, 13, 95]
[91, 95, 96, 21, 18]
[17, 68, 95, 37, 55]
[91, 95, 96, 21, 18]
[18, 95, 28, 59, 16]
[98, 97, 25, 2, 21]
[19, 95, 68, 4, 18]
[0, 31, 96, 25, 9]
[0, 31, 96, 25, 9]
[0, 68, 31, 59, 51]
[0, 68, 31, 59, 51]
[0, 31, 96, 25, 9]
[0, 31, 96, 25, 9]
[0, 31, 96, 25, 9]
[0, 31, 96, 25, 9]
[0, 31, 96, 25, 9]
[0, 31, 96, 25, 9]
[0, 31, 96, 25, 9]
[0, 31, 96, 25, 9]
[0, 31, 96, 25, 9]
[0, 31, 96, 25, 9]
[16, 48, 40, 18, 23]
[0, 68, 31, 59, 51]
[0, 68, 31, 59, 51]
[16, 91, 95]
[27, 74, 80, 86, 92]
[48, 77, 79, 91, 71]
[48, 77, 79, 91, 71]
[48, 77, 79, 91, 71]
[71, 34, 18, 77, 0]
[71, 34, 18, 77, 0]
[71, 34, 18, 77, 0]
[54, 96, 0, 34, 41]
[71, 34, 18, 77, 0]
[71, 34, 18, 77, 0]
[71, 34, 18, 77, 0]
[71, 34, 18, 77, 0]
[71, 34, 18, 77, 0]
[71, 34, 18, 77, 0]
[]
[37, 10, 70, 95, 16]
[91, 23, 47, 32, 50]
[28, 16

[]
[11, 72, 5, 33, 18]
[28, 13, 95, 42, 63]
[56, 98, 55, 70, 41]
[2, 29, 46, 59, 5]
[55, 64, 33, 3, 68]
[74]
[75, 42, 50, 91, 28]
[77, 43, 41, 72, 32]
[91, 9, 17, 32, 48]
[64, 22, 8, 99, 9]
[91, 94, 95, 21, 18]
[13, 16, 5, 21, 4]
[2, 25, 64, 58, 59]
[2, 25, 64, 58, 59]
[2, 25, 64, 58, 59]
[6, 28, 91, 42, 39]
[0, 15, 55, 25, 31]
[28, 99, 10, 91, 14]
[54, 0, 13, 15]
[71, 77, 23, 79, 34]
[56, 70, 98, 41, 55]
[56, 70, 98, 41, 69]
[56, 70, 98, 41, 69]
[50, 28, 47, 23]
[25, 5, 64, 11, 41]
[16]
[5]
[7, 91, 19, 89, 59]
[0, 96, 41, 83, 33]
[0, 34, 25, 73, 28]
[21, 68, 37, 64, 25]
[1, 79, 45, 24, 88]
[41, 97, 37, 69, 47]
[32, 42, 9, 28, 48]
[67, 81, 12]
[36, 61, 62, 15, 82]
[91, 9, 17, 32, 48]
[91, 9, 17, 32, 48]
[4, 23, 49, 19, 59]
[2, 25, 97, 64, 11]
[2, 55, 77, 98, 95]
[91, 9, 17, 32, 48]
[2, 25, 97, 64, 11]
[83, 56, 41, 70, 9]
[56, 98, 55, 70, 41]
[97, 9, 64, 85, 83]
[73, 31, 87, 30, 76]
[64, 8, 97, 46, 35]
[22, 25, 8, 55, 43]
[22, 43, 8, 58, 82]
[91, 18, 95, 59, 98]
[2, 55, 77, 98, 95]
[95,

[98, 97, 91, 41, 48]
[58, 30, 44, 46, 20]
[64, 8, 97, 46, 35]
[]
[40, 77, 50, 7, 91]
[73, 59]
[73, 59]
[18, 70]
[19, 31, 13, 39, 51]
[98, 70, 41, 19, 83]
[95, 98, 68, 64, 73]
[95, 98, 68, 64, 73]
[98, 97, 91, 41, 48]
[19, 25, 59, 21, 94]
[25, 2, 58, 64, 5]
[40, 48, 91]
[33, 18, 6, 95, 59]
[41, 77, 99, 40, 56]
[39, 91, 13]
[67, 82, 86, 62, 58]
[43]
[82, 91, 76, 32, 7]
[43, 42, 33, 16, 82]
[8, 48, 82, 5, 81]
[41, 83, 17, 97, 70]
[59, 32, 60, 13, 23]
[15, 48, 7, 50, 39]
[39, 77, 94, 99, 13]
[1, 79, 45, 24, 88]
[1, 79, 45, 24, 88]
[91, 95, 96, 21, 18]
[39, 72, 83, 91, 56]
[46, 36, 29, 9, 58]
[7, 41, 32, 42, 16]
[84, 59, 96, 34, 94]
[91, 48, 13, 47, 40]
[65, 66, 52, 87, 96]
[65, 66, 52, 87, 96]
[0, 84, 96, 41, 21]
[98, 97, 91, 41, 48]
[98, 97, 91, 41, 48]
[98, 97, 91, 41, 48]
[98, 97, 91, 41, 48]
[98, 97, 91, 41, 48]
[98, 97, 91, 41, 48]
[98, 97, 91, 41, 48]
[37, 10, 5]
[64, 10, 85, 59, 97]
[59, 90, 2, 61, 68]
[21, 91, 18, 68, 69]
[1, 79, 45, 24, 88]
[56, 70, 98, 41, 69]
[2, 25, 97, 64, 11]

[39, 77, 94, 99, 13]
[59, 29, 21, 25, 49]
[60]
[67, 46, 81]
[25, 5, 64, 11, 41]
[62, 36, 15, 30, 81]
[91, 8, 42, 50]
[91, 23, 47, 32, 50]
[59, 82, 62, 97, 2]
[17, 68, 95, 37, 55]
[70, 41, 3, 83, 21]
[38, 64, 67, 78, 58]
[]
[7, 89, 11, 59, 95]
[98, 70, 91, 48, 21]
[30, 90, 22, 14, 31]
[]
[36, 58, 62, 29, 85]
[5, 46, 85, 29, 36]
[50, 91, 4, 95, 2]
[83, 41, 17, 70, 5]
[68, 72, 93, 28, 15]
[78, 3, 36, 85, 64]
[97, 9, 64, 85, 83]
[56, 70, 98, 41, 69]
[]
[59, 2, 13, 68, 19]
[]
[25, 70, 69]
[40, 32, 48, 94, 18]
[90, 78, 5, 30, 16]
[64, 8, 97, 46, 35]
[90, 78, 5, 30, 16]
[28, 26]
[25, 5, 64, 11, 41]
[26, 69, 83, 96, 65]
[7, 47, 85]
[95, 68, 18, 19, 21]
[95, 68, 18, 41, 98]
[19, 25, 59, 21, 94]
[41, 72, 91, 68, 70]
[32, 39, 48, 33, 50]
[49, 33, 91, 59, 16]
[31, 65, 87, 64, 52]
[31, 65, 87, 64, 52]
[31, 65, 87, 64, 52]
[29, 82, 38, 11, 46]
[64, 2, 29, 95, 5]
[29, 82, 38, 11, 46]
[98, 97, 91, 41, 48]
[25, 58, 4, 6, 90]
[46, 64, 27, 85, 58]
[2, 25, 97, 64, 11]
[]
[91, 95, 96, 21, 18]
[1, 79, 45, 2

[1, 79, 45, 24, 88]
[38, 64, 67, 78, 58]
[72, 33, 91, 14]
[85, 7, 5, 58, 2]
[]
[85, 7, 5, 58, 2]
[85, 7, 5, 58, 2]
[1, 79, 45, 24, 88]
[49, 98, 18, 21, 68]
[91, 16, 42, 48, 94]
[7, 41, 10, 55, 90]
[9, 17, 37, 96, 6]
[65, 52, 87, 66, 96]
[0, 31, 96, 25, 9]
[48, 72, 18, 28, 96]
[59, 28, 26, 95, 39]
[1, 79, 45, 24, 88]
[1, 79, 45, 24, 88]
[]
[]
[32, 50, 6, 91, 4]
[32, 50, 6, 91, 4]
[91]
[46, 36, 9, 62, 58]
[96, 94, 34, 91, 93]
[95, 68, 18, 19, 21]
[91, 99, 48, 11, 72]
[91, 99, 48, 11, 72]
[91, 99, 48, 11, 72]
[91, 99, 48, 11, 72]
[91, 99, 48, 11, 72]
[91, 37, 16, 40, 50]
[59, 91, 19, 16, 68]
[59, 91, 19, 16, 68]
[16, 96, 7, 91, 6]
[43]
[91, 95, 42, 9, 48]
[0, 26, 34, 25, 28]
[0, 26, 34, 25, 28]
[13, 4, 73, 77, 50]
[30, 38, 46, 5, 89]
[64, 8, 97, 46, 35]
[64, 8, 97, 46, 35]
[55, 64, 17, 2, 98]
[46, 64, 58, 78, 67]
[11, 46, 8, 97, 35]
[86, 61, 6, 3, 67]
[89, 3]
[98, 97, 25, 2, 21]
[76]
[95, 68, 18, 41, 98]
[]
[]
[]
[95, 28, 16, 42, 11]
[95, 28, 16, 42, 11]
[29, 82, 38, 11, 46]
[95, 77, 4, 2

[9, 18, 55, 41, 37]
[32, 40, 41, 7, 48]
[91, 37, 16, 40, 50]
[32, 39, 75]
[10, 50]
[1, 79, 45, 24, 88]
[]
[10, 50]
[59, 28, 26, 95, 39]
[82, 62, 78, 81, 86]
[8, 48, 82, 5, 81]
[8, 48, 82, 5, 81]
[1, 79, 45, 24, 88]
[79, 34, 50, 39, 55]
[1, 79, 45, 24, 88]
[1, 79, 45, 24, 88]
[81, 58, 46]
[77, 39, 42, 59]
[73, 51, 18, 83, 91]
[59, 28, 26, 95, 39]
[59, 51, 47]
[0, 37, 17, 91, 33]
[0, 31, 96, 25, 9]
[33, 95, 16, 42, 19]
[28, 13, 96, 91, 16]
[56, 70, 98, 41, 55]
[91, 42]
[61, 99, 38, 46, 36]
[98, 19, 70, 56, 40]
[56, 70, 98, 41, 69]
[2, 25, 97, 64, 11]
[2, 25, 97, 64, 11]
[22, 64, 8, 37, 2]
[25, 64, 90, 37, 83]
[25, 64, 90, 37, 83]
[89, 3, 7, 48, 50]
[13, 91, 18, 68, 4]
[89, 3, 7, 48, 50]
[31, 17, 77, 10, 73]
[8, 50, 91, 94, 96]
[91, 25, 50, 63, 3]
[91, 25, 50, 63, 3]
[91, 9, 17, 32, 48]
[91, 9, 17, 32, 48]
[91, 9, 17, 32, 48]
[91, 9, 17, 32, 48]
[]
[28, 50, 7, 32, 91]
[1, 79, 45, 24, 88]
[71, 34, 18, 77, 0]
[7, 97, 28, 70, 32]
[56, 70, 98, 41, 69]
[13, 77, 83, 90, 7]
[48, 72, 18, 28, 96]


[26, 73, 84, 90, 92]
[1, 79, 45, 24, 88]
[48, 91, 32, 42]
[48, 91, 32, 42]
[48, 91, 32, 42]
[59]
[90, 62, 36, 78, 15]
[1, 79, 45, 24, 88]
[6]
[64, 22, 8, 99, 9]
[36, 89, 30, 81, 35]
[77]
[94, 48, 50, 91]
[85, 10, 99, 30, 62]
[85, 10, 99, 30, 62]
[91, 48, 7, 15, 28]
[4, 25, 83, 73, 37]
[4, 25, 83, 73, 37]
[41, 72, 91, 68, 70]
[16, 5, 47, 7, 39]
[91, 72, 69, 98, 48]
[74]
[28, 15, 99, 48, 76]
[15, 7, 32, 91, 48]
[48, 91, 94]
[29, 43, 82, 5, 42]
[59, 18, 11, 21, 30]
[2, 25, 59, 82, 42]
[13, 91, 18, 68, 4]
[83, 41, 25, 68, 70]
[95, 68, 18, 19, 21]
[2, 29, 59, 5, 58]
[41, 72, 91, 68, 70]
[41, 72, 91, 68, 70]
[9, 55, 95, 21, 49]
[73, 71, 31, 93, 90]
[71, 34, 18, 77, 0]
[91, 18, 95, 59, 98]
[98, 97, 25, 2, 21]
[67, 58, 61, 2, 5]
[4, 48, 77, 0, 33]
[48, 73, 59]
[18, 2, 59, 13, 91]
[18, 2, 59, 13, 91]
[]
[63, 3, 86, 46, 81]
[25, 82, 72, 78, 30]
[6, 82, 59, 75, 61]
[1, 79, 45, 24, 19]
[48, 2]
[91, 95, 96, 21, 18]
[68, 21, 98, 70, 49]
[91, 95, 96, 21, 18]
[29, 64, 46, 5, 97]
[61, 75, 40]
[58, 30, 

In [49]:
metrics.mapk([[l] for l in t2["hotel_cluster"]], preds, k=5)

0.22662427651422518

<b> Generating better predictions for your Kaggle submission </b>

The forums are very important in Kaggle, and can often help you find nuggets of information that will let you boost your score. The Expedia competition is no exception. <a href ="https://www.kaggle.com/c/expedia-hotel-recommendations/forums/t/20345/data-leak"> <b>This post</b></a> details a data leak that allows you to match users in the training set from the testing set using a set of columns including user_location_country, and user_location_region.

We’ll use the information from the post to match users from the testing set back to the training set, which will boost our score. Based on the forum thread, its okay to do this, and the competition won’t be updated as a result of the leak.


<b> Finding matching users</b> 

The first step is to find users in the training set that match users in the testing set.

In order to do this, we need to:
<ul>
<li> Split the training data into groups based on the match columns.
<li> Loop through the testing data.
<li> Create an index based on the match columns.
<li> Get any matches between the testing data and the training data using the groups.
</ul>

In [51]:
match_cols = ['user_location_country', 'user_location_region', 'user_location_city', 'hotel_market', 'orig_destination_distance']

groups = t1.groupby(match_cols)
    
def generate_exact_matches(row, match_cols):
    index = tuple([row[t] for t in match_cols])
    try:
        group = groups.get_group(index)
    except Exception:
        return []
    clus = list(set(group.hotel_cluster))
    return clus

exact_matches = []
for i in range(t2.shape[0]):
    exact_matches.append(generate_exact_matches(t2.iloc[i], match_cols))

At the end of this loop, we’ll have a list of lists that contain any exact matches between the training and the testing sets. However, there aren’t that many matches. To accurately evaluate error, we’ll have to combine these predictions with our earlier predictions. Otherwise, we’ll get a very low accuracy value, because most rows have empty lists for predictions.

<b> Combining predictions</b>

We can combine different lists of predictions to boost accuracy. Doing so will also help us see how good our exact match strategy is. To do this, we’ll have to:

<ul>

<li> Combine exact_matches, preds, and most_common_clusters.
<li> Only take the unique predictions, in sequential order, using the f5 function from <a href ="https://www.peterbe.com/plog/uniqifiers-benchmark"> <b> here </b> </a>.
<li> Ensure we have a maximum of 5 predictions for each row in the testing set.
</ul>


In [53]:
def f5(seq, idfun=None): 
    if idfun is None:
        def idfun(x): return x
    seen = {}
    result = []
    for item in seq:
        marker = idfun(item)
        if marker in seen: continue
        seen[marker] = 1
        result.append(item)
    return result
    
full_preds = [f5(exact_matches[p] + preds[p] + most_common_clusters)[:5] for p in range(len(preds))]
metrics.mapk([[l] for l in t2["hotel_cluster"]], full_preds, k=5)

0.26818292981168995

<b> Generating predictions file </b>


In [63]:
print(t2.columns.values)
write_p = [" ".join([str(l) for l in p]) for p in full_preds]

['date_time' 'site_name' 'posa_continent' 'user_location_country'
 'user_location_region' 'user_location_city' 'orig_destination_distance'
 'user_id' 'is_mobile' 'is_package' 'channel' 'srch_ci' 'srch_co'
 'srch_adults_cnt' 'srch_children_cnt' 'srch_rm_cnt' 'srch_destination_id'
 'srch_destination_type_id' 'is_booking' 'cnt' 'hotel_continent'
 'hotel_country' 'hotel_market' 'hotel_cluster' 'year' 'month']


In [67]:
write_frame = ["{0},{1}".format(t2["user_id"].iloc[i], write_p[i]) for i in range(len(full_preds))]
write_frame = ["id,hotel_clusters"] + write_frame
with open("predictions.csv", "w+") as f:
    f.write("\n".join(write_frame))

<b> Generating the predictions for actual train and test files </b>

Now, we repeat the process we did for the t1 and t2 by replacing the t1 variable by train and t2 varaible by test data.
The whole block of code for srch_destination_id based prediction looks like:


In [58]:
print(train.columns.values)
print(test.columns.values)

['date_time' 'site_name' 'posa_continent' 'user_location_country'
 'user_location_region' 'user_location_city' 'orig_destination_distance'
 'user_id' 'is_mobile' 'is_package' 'channel' 'srch_ci' 'srch_co'
 'srch_adults_cnt' 'srch_children_cnt' 'srch_rm_cnt' 'srch_destination_id'
 'srch_destination_type_id' 'is_booking' 'cnt' 'hotel_continent'
 'hotel_country' 'hotel_market' 'hotel_cluster' 'year' 'month']
['id' 'date_time' 'site_name' 'posa_continent' 'user_location_country'
 'user_location_region' 'user_location_city' 'orig_destination_distance'
 'user_id' 'is_mobile' 'is_package' 'channel' 'srch_ci' 'srch_co'
 'srch_adults_cnt' 'srch_children_cnt' 'srch_rm_cnt' 'srch_destination_id'
 'srch_destination_type_id' 'hotel_continent' 'hotel_country'
 'hotel_market']


In [104]:
match_cols = ["srch_destination_id"]
cluster_cols = match_cols + ['hotel_cluster']
groups = train.groupby(cluster_cols)
top_clusters = {}
for name, group in groups:
    clicks = len(group.is_booking[group.is_booking == False])
    bookings = len(group.is_booking[group.is_booking == True])
    
    score = bookings + .15 * clicks
    
    clus_name = make_key(name[:len(match_cols)])
    if clus_name not in top_clusters:
        top_clusters[clus_name] = {}
    top_clusters[clus_name][name[-1]] = score

import operator
cluster_dict = {}
for n in top_clusters:
    tc = top_clusters[n]
    top = [l[0] for l in sorted(tc.items(), key=operator.itemgetter(1), reverse=True)[:5]]
    cluster_dict[n] = top

#making predictions
preds = []
for index, row in test.iterrows():
    key = make_key([row[m] for m in match_cols])
    if key in cluster_dict:
        preds.append(cluster_dict[key])
    else:
        preds.append([])

In [97]:
print(len(preds))
print(len(exact_matches))
print(len(list(top_clusters.keys())))

2528243
8178
59455


In [74]:
# exact_matches and preds are not of same length so we select the combination based on the value of p
full_preds = [f5((exact_matches[p] + preds[p] + most_common_clusters) if p<len(exact_matches) else (preds[p] + most_common_clusters))[:5] for p in range(len(preds))]

In [75]:
write_p = [" ".join([str(l) for l in p]) for p in full_preds]

In [83]:
print(len(test["user_id"]))
print(len(write_p))
print(len(test["user_id"].unique()))

2528243
2528243
1181577


In [99]:
#write_frame = ["{0},{1}".format(test["user_id"].iloc[i], write_p[i]) for i in range(len(test["user_id"]))]
write_frame = []
#test has duplicate user_id columns but this is not allowed in submission, so we filter them out without destroying the order
for i in range(len(test["user_id"])):
    userid = test["user_id"].iloc[i]
    #write_frame.append("{0},{1}".format(test["user_id"].iloc[i], write_p[i]))
    write_frame.append("{0},{1}".format(i, write_p[i]))
print(len(write_frame))  

2528243


In [100]:
write_frame = ["id,hotel_cluster"] + write_frame
with open("predictions_1.csv", "w+") as f:
    f.write("\n".join(write_frame))

<b> Generating predictions using matching users in train and test data </b> 


In [101]:
match_cols = ['user_location_country', 'user_location_region', 'user_location_city', 'hotel_market', 'orig_destination_distance']

groups = train.groupby(match_cols)
    
def generate_exact_matches(row, match_cols):
    index = tuple([row[t] for t in match_cols])
    try:
        group = groups.get_group(index)
    except Exception:
        return []
    clus = list(set(group.hotel_cluster))
    return clus

exact_matches = []
for i in range(test.shape[0]):
    exact_matches.append(generate_exact_matches(test.iloc[i], match_cols))

#combining predictions
def f5(seq, idfun=None): 
    if idfun is None:
        def idfun(x): return x
    seen = {}
    result = []
    for item in seq:
        marker = idfun(item)
        if marker in seen: continue
        seen[marker] = 1
        result.append(item)
    return result

In [ ]:
# exact_matches and preds are not of same length so we select the combination based on the value of p
full_preds = [f5((exact_matches[p] + preds[p] + most_common_clusters) if p<len(exact_matches) else (preds[p] + most_common_clusters))[:5] for p in range(len(preds))]
mapk([[l] for l in test["hotel_cluster"]], full_preds, k=5)

In [102]:
write_p = [" ".join([str(l) for l in p]) for p in full_preds]
write_frame = []
#test has duplicate user_id columns but this is not allowed in submission, so we filter them out without destroying the order
for i in range(len(test["user_id"])):
    userid = test["user_id"].iloc[i]
    #write_frame.append("{0},{1}".format(test["user_id"].iloc[i], write_p[i]))
    write_frame.append("{0},{1}".format(i, write_p[i]))
print(len(write_frame))  

2528243


<b> Generating Submission file </b>

In [103]:
write_frame = ["id,hotel_cluster"] + write_frame
with open("predictions_2.csv", "w+") as f:
    f.write("\n".join(write_frame))

<b> Predictions based on Most Frequent Hotel</b>

In [98]:
"""
from csv import DictReader
from collections import defaultdict
from datetime import datetime

start = datetime.now()

def get_top5(d):
    return " ".join(sorted(d, key=d.get, reverse=True)[:5])

destination_clusters = defaultdict(lambda: defaultdict(int))

for i, row in enumerate(DictReader(open("train.csv"))):
	destination_clusters[row["srch_destination_id"]][row["hotel_cluster"]] += 1
	if i % 1000000 == 0:
		print("%s\t%s"%(i, datetime.now() - start))

most_frequent = defaultdict(str)

for k in destination_clusters:
	most_frequent[k] = get_top5(destination_clusters[k])

with open("pred_sub.csv", "w") as outfile:
	outfile.write("id,hotel_cluster\n")
	for i, row in enumerate(DictReader(open("test.csv"))):
		outfile.write("%d,%s\n"%(i,most_frequent[row["srch_destination_id"]]))
"""

0	0:00:20.305560
1000000	0:00:37.279053
2000000	0:00:52.729882
3000000	0:01:08.141346
4000000	0:01:23.711879
5000000	0:01:39.560328
6000000	0:01:55.423821
7000000	0:02:10.880847
8000000	0:02:26.287971
9000000	0:02:41.647730
10000000	0:02:57.079803
11000000	0:03:12.538132
12000000	0:03:28.064009
13000000	0:03:43.905353
14000000	0:03:59.655803
15000000	0:04:15.331883
16000000	0:04:31.192137
17000000	0:04:46.715946
18000000	0:05:02.210153
19000000	0:05:17.589902
20000000	0:05:33.120016
21000000	0:05:48.949862
22000000	0:06:04.366883
23000000	0:06:20.079414
24000000	0:06:35.841360
25000000	0:06:51.490396
26000000	0:07:07.191595
27000000	0:07:23.423269
28000000	0:07:39.885516
29000000	0:07:55.643842
30000000	0:08:11.336734
31000000	0:08:26.863238
32000000	0:08:42.376971
33000000	0:08:57.882157
34000000	0:09:13.331494
35000000	0:09:28.817095
36000000	0:09:44.974422
37000000	0:10:00.649368


<b>References </b>

A post from Vik Paruchuri  at https://www.dataquest.io/blog/kaggle-tutorial/